In [1]:
import cv2
import numpy as np

In [2]:
class FrameBuffer:
    def __init__(self, delta = 5, period = 2000):
        self.data = []
        self.delta = delta
        self.period = period
        self.maxLen = period/delta
        self.len = 0
    def store(self, frame):
        self.data.append(frame)
        self.len+=1
        if self.len>self.maxLen:
            self.data = self.data[1:]
            self.len-=1
    def express(self, distance):
        if distance//self.delta>self.len-2:
            out = self.data[0], self.data[-1]
        else:
            out = self.data[-1-distance//self.delta], self.data[-1]
        return out
    

In [3]:
class DetectedObject:#thread unsafe private class
    def __init__(self, x1, y1, x2, y2, time):
        elf.center = (np.array([x1, y1], dtype='float64')+ np.array([x2, y2], dtype='float64'))/2#[(x1+x2)/2, (y1+y2)/2]
        self.xyxy_story =[[[self.x1, self.y1],[self.x2, self.y2]]]
        self.center_story = [self.center]
        self.time = [time]
    def update(self, x1, y1, x2, y2, time):
        self.x1, self.y1, self.x2, self.y2 = x1, y1, x2, y2
        self.center = (np.array([x1, y1])+np.array([x2, y2]))/2
        self.xyxy_story.append([[self.x1, self.y1],[self.x2, self.y2]])
        self.center_story.append(self.center)
        self.time.append(time)
    def print(self):
        print('xyxy       :', self.x1, self.y1, self.x2, self.y2)

In [4]:
class ObjectStore:
    def __init__(self):
        self.data = [] #iteration on it without mutex is thread unsafe
        self.thresh = 0.5
        self.time = None
    def print(self):
        k = 0
        for i in self.data:
            print('######', k, '######')
            k+=1
            i.print()
        
        print('######END######')
    def find(self, x1, y1, x2, y2):#private function
        r = None
        for i in self.data:
            xx1 = max(x1, i.x1)
            xx2 = min(x2, i.x2)
            yy1 = max(y1, i.y1)
            yy2 = min(y2, i.y2)
            xxx1 = min(x1, i.x1)
            xxx2 = max(x2, i.x2)
            yyy1 = min(y1, i.y1)
            yyy2 = max(y2, i.y2)
            ww = xx2-xx1
            hh = yy2-yy1
            w = xxx2-xxx1
            h = yyy2-yyy1
            #print(ww, hh, w, h, ww*hh/(w*h))
            if ww<=0 or hh<=0:
                continue
            if ww*hh/(w*h)>self.thresh:
                return i, True
        return None, False
    def store(self, x1, y1, x2, y2, time):
        self.time = time
        i, F = self.find(x1, y1, x2, y2)
        if F:
            i.update(x1, y1, x2, y2, time)
        else:
            i = DetectedObject(x1, y1, x2, y2, time)
            self.data.append(i)

In [5]:
def prepareFrame(frame, crop=False, crop_v=[0, 200, 0, 800], ksize_val=5):
    if crop:
        frame = frame[crop_v[0]:crop_v[1], crop_v[2]:crop_v[3]]
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    frame = cv2.GaussianBlur(src=frame, ksize=(ksize_val, ksize_val), sigmaX=0)
    return frame
def compareFrames(frame1, frame2, dilate=False, dil_val=5, thresh_val=30):
    diff = cv2.absdiff(src1=frame1, src2=frame2)
    if dilate:
        kernel = np.ones((dil_val, dil_val))
        diff = cv2.dilate(diff, kernel, 1)
    diff = cv2.threshold(src=diff, thresh=thresh_val, maxval=255, type=cv2.THRESH_BINARY)[1]
    contours, _ = cv2.findContours(image=diff, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
    return diff, contours
def filterContours(contours, area_val=5, wh_val=10):
    con_f = []
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < area_val:
            continue
        (x, y, w, h) = cv2.boundingRect(contour)
        if w*h<wh_val:
            continue
        con_f.append(contour)
    return con_f
def drawContours(im, contours, crop=False, crop_v=[0, 200, 0, 800], color=(0, 255, 255), thickness=1, with_rectangles=False, color_rectangles=(255,0,255), thickness_rectangles=2):
    if crop and (crop_v[0]!=0 or crop_v[2]!=0):
        contours = contours.copy()
        for contour in contours:
            contour += [crop_v[0], crop_v[2]]
    cv2.drawContours(image=im, contours=contours, contourIdx=-1, color=color, thickness=thickness, lineType=cv2.LINE_AA)
    if with_rectangles:
        for contour in contours:
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(img=im, pt1=(x, y), pt2=(x + w, y + h), color=color_rectangles, thickness=thickness_rectangles)

In [6]:
def compare_two_frames(prev, curr, out, objectStore, area_val=50, wh_val=100,  crop=False, crop_v=[0, 200, 0, 800], draw=True, color=(0, 255, 255), color_super=(0, 0, 0), thickness=1, with_rectangles=False, color_rectangles=(255,0,0), thickness_rectangles=2):
    prev, curr = prepareFrame(prev, crop=crop, crop_v=crop_v, ksize_val=3), prepareFrame(curr, crop=crop, crop_v=crop_v, ksize_val=3)
    diff, cnt = compareFrames(prev, curr, dilate=False, dil_val=5, thresh_val=25)
    con_f = filterContours(cnt, area_val=area_val, wh_val=wh_val)
    if draw:
        drawContours(out, con_f, crop=crop, crop_v=crop_v, color=color, thickness=thickness, with_rectangles=with_rectangles, color_rectangles=color_rectangles, thickness_rectangles=thickness_rectangles)
    return con_f

In [7]:
def drawObjectRectangles(pic, objectStore):
    for b in objectStore.data:
        cv2.rectangle(pic, (int(b.x1), int(b.y1)), (int(b.x2), int(b.y2)))

In [8]:
import matplotlib.pyplot as plt


source='VideoMaterials/pen2.mp4'
source='1.webm'
view_img=True  # show results

######################################################################
skip = 10
delta_compare = 20
assert(skip > 0)
crop_v = [400, 800, 900, 1920]
crop = False
crop2 = False
crop_v2 = [0, 170, 0, 1000]
######################################################################

frameBuffer = FrameBuffer(delta=skip, period=delta_compare+skip)
objectStore = ObjectStore()

cap = cv2.VideoCapture(source)

# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        out_pic = frame.copy()
        frameBuffer.store(frame)
        prev, curr = frameBuffer.express(20)
        con_f = compare_two_frames(prev, curr, out_pic, objectStore, area_val=600, wh_val=1200, crop=crop2, crop_v=crop_v2, draw=True, color=(0, 255, 255), thickness=10, with_rectangles=True, color_rectangles=(255,0,255), thickness_rectangles=8)
        
        prev, curr = prepareFrame(prev, crop=crop, crop_v=crop_v, ksize_val=3), prepareFrame(curr, crop=crop, crop_v=crop_v, ksize_val=3)
        diff, cnt = compareFrames(prev, curr, dilate=False, dil_val=5, thresh_val=25)
        #con_f = filterContours(cnt, area_val=50, wh_val=100)
        #drawContours(out_pic, con_f, crop=crop, crop_v=crop_v, color=color, thickness=thickness, with_rectangles=with_rectangles, color_rectangles=color_rectangles, thickness_rectangles=thickness_rectangles)
        cv2.imshow('Diff',diff)
        
            
        drawObjectRectangles(out_pic, objectStore)
        # Display the resulting frame
        cv2.imshow('Frame',out_pic)

        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    # Break the loop
    else: 
        break

# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()
